In [1]:
import os
import requests
import zipfile
import io
import pandas as pd

# Download, extract, and load .dta file into a DataFrame

In [2]:
# Function to download, extract, and load .dta file into a DataFrame
def load_dta_from_zip(url, year):
    print(f"Processing dataset for year {year}...")
    response = requests.get(url)
    zip_file = zipfile.ZipFile(io.BytesIO(response.content))
    zip_file.extractall(f"extracted_files_{year}")

    # List all extracted files
    extracted_files = zip_file.namelist()
    print(f"Extracted files for {year}: {extracted_files}")

    # Identify .dta files
    dta_files = [file for file in extracted_files if file.endswith('.dta')]
    print(f".dta files found for {year}: {dta_files}")

    if dta_files:
        dta_file_path = os.path.join(f"extracted_files_{year}", dta_files[0])
        try:
            df = pd.read_stata(dta_file_path)
            print(f"DataFrame for {year} loaded successfully.")
            return df
        except ValueError as e:
            print(f"Error loading .dta file for {year}: {e}")
            return None
    else:
        print(f"No .dta files found for {year}.")
        return None

# URLs for datasets

In [3]:
urls = {
    2017: "https://proyectos.inei.gob.pe/iinei/srienaho/descarga/STATA/615-Modulo1332.zip",
    2018: "https://proyectos.inei.gob.pe/iinei/srienaho/descarga/STATA/650-Modulo1468.zip",
    2019: "https://proyectos.inei.gob.pe/iinei/srienaho/descarga/STATA/701-Modulo1547.zip"
}

# Loading the datasets

In [4]:
df_2017 = load_dta_from_zip(urls[2017], 2017)
df_2018 = load_dta_from_zip(urls[2018], 2018)
df_2019 = load_dta_from_zip(urls[2019], 2019)

Processing dataset for year 2017...
Extracted files for 2017: ['615-Modulo1332/', '615-Modulo1332/01_CUESTIONARIO_PEQUEÑOS_MEDIANOS_2017.pdf', '615-Modulo1332/02_CUESTIONARIO_ESTRATO_ESPECIAL_2017.pdf', '615-Modulo1332/18_Cap1100.dta', '615-Modulo1332/Diccionario_Datos_18_CAP1100.pdf']
.dta files found for 2017: ['615-Modulo1332/18_Cap1100.dta']
DataFrame for 2017 loaded successfully.
Processing dataset for year 2018...
Extracted files for 2018: ['650-Modulo-1468/', '650-Modulo-1468/01_CUESTIONARIO_PEQUEÑOS_MEDIANOS_2018.pdf', '650-Modulo-1468/02_CUESTIONARIO_ESTRATO_ESPECIAL_2018.pdf', '650-Modulo-1468/19_Cap1100.dta', '650-Modulo-1468/Diccionario_Datos_19_Cap1100.pdf']
.dta files found for 2018: ['650-Modulo-1468/19_Cap1100.dta']
DataFrame for 2018 loaded successfully.
Processing dataset for year 2019...
Extracted files for 2019: ['701-Modulo1547/', '701-Modulo1547/01_CUESTIONARIO_PEQUEÑOS_MEDIANOS_2019.pdf', '701-Modulo1547/02_CUESTIONARIO_ESTRATO_ESPECIAL_2019.pdf', '701-Modulo1547

# Column name mapping based on the PDF dictionary

In [5]:
# Column mappings for each dataset
column_mapping_2017 = {
    "ANIO": "year",
    "CCDD": "region_id",
    "NOMBREDD": "region",
    "CCPP": "province_id",
    "NOMBREPV": "province",
    "CCDI": "district_id",
    "NOMBREDI": "district",
    "CONGLOMERADO": "conglomerate",
    "NSELUA": "agricultural_unit_selection_number",
    "UA": "agricultural_unit_mumber",
    "ESTRATO": "stratum_type",
    "RESFIN": "survey_final_result",
    "REGION": "ntural_region",
    "DOMINIO": "geographic_domain",
    "FACTOR": "expansion_factor",
    "CODIGO": "identification_code",
    "P102_1": "agriculture_12_months",
    "P102_2": "livestock_12_months",
    "P1100": "order_number",
    "P1102": "relationship_agricultural_producer",
    "P1103": "gender",
    "P1104_A": "age_years",
    "P1104_B": "age_months",
    "P1105": "education_level",
    "P1106": "language_childhood",
    "P1107": "Participates in Agricultural Activities",
    "P1108_1": "disability_movement",
    "P1108_2": "disability_visual",
    "P1108_3": "disability_communication",
    "P1108_4": "disability_hearing",
    "P1108_5": "disability_understanding",
    "P1108_6": "disability_relationships",
    "P1109": "ethnic_group",
    "P1109A": "engaged_on_other_income",
    "P1109B": "main_other_income",
    "P1110": "lives_at_the_farm",
    "OMICAP1100": "chapter_1100_omission",
}

column_mapping_2018 = column_mapping_2017.copy()
column_mapping_2018.update({
    "P101A": "producer_legal_status",
    "P102_3": "no_activity_12_months",
})

column_mapping_2019 = column_mapping_2018.copy()

In [6]:
# Function to standardize column names
def standardize_columns(df, mapping):
    """
    Standardizes column names in the dataset using a unified mapping and retains only columns present in the DataFrame.

    Parameters:
    df (DataFrame): The dataset to be standardized.
    mapping (dict): The column mapping dictionary.

    Returns:
    DataFrame: The dataset with standardized column names.
    """
    existing_columns = {col: mapping[col] for col in df.columns if col in mapping}
    return df.rename(columns=existing_columns, inplace=False)

In [7]:
# Standardize all datasets
df_2017_standardized = standardize_columns(df_2017, column_mapping_2017)
df_2018_standardized = standardize_columns(df_2018, column_mapping_2018)
df_2019_standardized = standardize_columns(df_2019, column_mapping_2019)

In [8]:
# Align columns across datasets
all_columns_ordered = list(column_mapping_2017.values())

for col in df_2018_standardized.columns:
    if col not in all_columns_ordered:
        all_columns_ordered.append(col)

for col in df_2019_standardized.columns:
    if col not in all_columns_ordered:
        all_columns_ordered.append(col)

df_2017_aligned = df_2017_standardized.reindex(columns=all_columns_ordered)
df_2018_aligned = df_2018_standardized.reindex(columns=all_columns_ordered)
df_2019_aligned = df_2019_standardized.reindex(columns=all_columns_ordered)

In [9]:
# Add 'Year' column to identify datasets
df_2017_aligned['Year'] = 2017
df_2018_aligned['Year'] = 2018
df_2019_aligned['Year'] = 2019

In [10]:
# Combine datasets and add a unique_id column starting at 1
combined_df = pd.concat([df_2017_aligned, df_2018_aligned, df_2019_aligned], ignore_index=True)

# Add unique_id as the first column
combined_df.insert(0, "unique_id", range(1, len(combined_df) + 1))


C:\Users\jcbur\AppData\Local\Temp\ipykernel_3212\369379374.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([df_2017_aligned, df_2018_aligned, df_2019_aligned], ignore_index=True)


C:\Users\jcbur\AppData\Local\Temp\ipykernel_3212\369379374.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([df_2017_aligned, df_2018_aligned, df_2019_aligned], ignore_index=True)
C:\Users\jcbur\AppData\Local\Temp\ipykernel_3212\369379374.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([df_2017_aligned, df_2018_aligned, df_2019_aligned], ignore_index=True)


In [11]:
# Save the combined dataset to a CSV file
### output_path = r"C:\Users\jcbur\OneDrive - peruvianbusinesscouncil.com\csv files\#14_Producer_Geo_Spatial_Analysis\ena_producer_profile_2017_2019.csv"
### combined_df.to_csv(output_path, index=False)

In [12]:
print(combined_df)

        unique_id    year region_id  region province_id  \
0               1  2017.0        24  TUMBES          02   
1               2  2017.0        24  TUMBES          02   
2               3  2017.0        24  TUMBES          02   
3               4  2017.0        24  TUMBES          02   
4               5  2017.0        24  TUMBES          02   
...           ...     ...       ...     ...         ...   
280414     280415  2019.0        23   TACNA          01   
280415     280416  2019.0        23   TACNA          01   
280416     280417  2019.0        23   TACNA          01   
280417     280418  2019.0        23   TACNA          01   
280418     280419  2019.0        23   TACNA          01   

                     province district_id             district conglomerate  \
0       CONTRALMIRANTE VILLAR          03  CANOAS DE PUNTA SAL        00001   
1       CONTRALMIRANTE VILLAR          03  CANOAS DE PUNTA SAL        00001   
2       CONTRALMIRANTE VILLAR          03  CANOAS DE P

In [13]:
observations = {}
for col in combined_df.columns:
    observations[col] = {
        'total_count': combined_df[col].size,
        'nan_count': combined_df[col].isna().sum(),
        'dtype': combined_df[col].dtype,
        'mean': combined_df[col].mean() if combined_df[col].dtype in ['int64', 'float64'] else None,
        'unique_count': combined_df[col].nunique(),
        'categories': combined_df[col].value_counts().head(10).to_dict() if combined_df[col].dtype == 'object' or combined_df[col].nunique() <= 10 else None
    }

for feature, details in observations.items():
    print(f"Feature: {feature}")
    for key, value in details.items():
        print(f"  {key}: {value}")
    print()

Feature: unique_id
  total_count: 280419
  nan_count: 0
  dtype: int64
  mean: 140210.0
  unique_count: 280419
  categories: None

Feature: year
  total_count: 280419
  nan_count: 0
  dtype: float64
  mean: 2017.9759181795812
  unique_count: 3
  categories: {2017.0: 96382, 2018.0: 94408, 2019.0: 89629}

Feature: region_id
  total_count: 280419
  nan_count: 0
  dtype: object
  mean: None
  unique_count: 25
  categories: {'20': 14912, '13': 14339, '14': 14205, '01': 13897, '02': 13767, '16': 13728, '06': 13720, '08': 13225, '10': 13169, '22': 12576}

Feature: region
  total_count: 280419
  nan_count: 0
  dtype: object
  mean: None
  unique_count: 29
  categories: {'PIURA': 14912, 'LA LIBERTAD': 14339, 'LAMBAYEQUE': 14205, 'AMAZONAS': 13897, 'ANCASH': 13767, 'LORETO': 13728, 'CAJAMARCA': 13720, 'CUSCO': 13225, 'HUANCAVELICA': 12311, 'PUNO': 11802}

Feature: province_id
  total_count: 280419
  nan_count: 0
  dtype: object
  mean: None
  unique_count: 20
  categories: {'01': 70928, '02': 43